In [ ]:
#librerías uso
import os
import openai
from pymed import PubMed
import pandas as pd

#credenciales API OpenAI


ModuleNotFoundError: ignored

In [ ]:
def getAnswer(conversation, temperature=1, top_p=1, maxT=800):
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo",  # Elige el motor adecuado
        messages=conversation,
        temperature=temperature,  # Controla la creatividad de las respuestas
        top_p = top_p,
        max_tokens = maxT
    )
    return response

In [ ]:
conversation = [{"role": "user", "content": "hola"}]
response = getAnswer(conversation)

In [ ]:
pubmed = PubMed(tool="PubMedSearcher", email="granadoslm1@gmail.com")

## PUT YOUR SEARCH TERM HERE ##
search_term = "Tuberculosis"
results = pubmed.query(search_term, max_results=500)
articleList = []
articleInfo = []

for article in results:
    articleDict = article.toDict()
    articleList.append(articleDict)

for article in articleList:
    pubmedId = article['pubmed_id'].partition('\n')[0]
    articleInfo.append({
        u'pubmed_id': pubmedId,
        u'title': article['title'],
        u'keywords': article['keywords'],
        u'journal': article['journal'],
        u'abstract': article['abstract'],
        u'conclusions': article['conclusions'],
        u'methods': article['methods'],
        u'results': article['results'],
        u'copyrights': article['copyrights'],
        u'doi': article['doi'],
        u'publication_date': article['publication_date'],
        u'authors': article['authors']
    })

articlesPD = pd.DataFrame.from_dict(articleInfo)
#export_csv = articlesPD.to_csv(r'C:\Users\Patri\Documents\export_dataframe.csv', index=None, header=True)


In [ ]:
articlesPD.columns

Index(['pubmed_id', 'title', 'keywords', 'journal', 'abstract', 'conclusions',
       'methods', 'results', 'copyrights', 'doi', 'publication_date',
       'authors'],
      dtype='object')

In [ ]:
abstractText = articlesPD.abstract
print(abstractText)

0      Cancer is a leading cause of death globally an...
1      By illuminating key 6-azasteroid-protein inter...
2      From 2018-2021 the TB Reduction through Expand...
3      Quantification of recurrence risk following su...
4      Systematic Review.\nTo review the literature f...
                             ...                        
495    Technology including artificial intelligence (...
496    To determine the yield of cervical mediastinos...
497    Herein, we report the design and synthesis of ...
498    Pyrazinamide (PZA) and fluoroquinolone (FQ), p...
499    We read with interest the systematic review by...
Name: abstract, Length: 500, dtype: object


In [ ]:
enfermedades = []
iterations = 0
initialPrompt = """
Te voy a dar una serie de resumenes de [documentos] y tu tarea es decirme sobre que enfermedad están hablando.
Tienes que proceder de la siguiente manera:
    - Tienes que leer cada fragmento e identificar la enfermedad sobre la que están hablando en el [documento]
    - Cuando termines de leer cada fragmento tienes que decir solamente la enfermedad sobre la que hablan.
    - En caso de que no sepas sobre que enfermedad hablan tienes que decir solamente: NO_IDENTIFICADO

El fragmento de [documento] donde buscar la enfermedad de la que habla es el siguiente. [Documento] = """
for item in abstractText:
    conversation = [{"role": "user", "content": initialPrompt+str(item)}]
    try:
         response = getAnswer(conversation)
         enfermedades.append(response)

    except Exception as e:
        print(f"Error al obtener respuesta para '{item}': {str(e)}")

    if iterations == 20:
        break

    iterations += 1



In [ ]:
for i in range(0, len(enfermedades)):
     try:
            print("\n*****************************************************************************************************************************\n","Titulo: ", i+1, "\n\n", enfermedades[i]["choices"][0]["message"]["content"])
     except Exception as e:
        print("*****************************************************************************************************************************\n")
        print(f"\033[1;31m Error al volcar información en el elemento número: '{i+1}': {str(e)} \033[0m\n")


*****************************************************************************************************************************
 Titulo:  1 

 NO_IDENTIFICADO

*****************************************************************************************************************************
 Titulo:  2 

 NO_IDENTIFICADO

*****************************************************************************************************************************
 Titulo:  3 

 Tuberculosis (TB)

*****************************************************************************************************************************
 Titulo:  4 

 Enfermedad: Tuberculosis multidrogorresistente (MDR/RR)

*****************************************************************************************************************************
 Titulo:  5 

 Enfermedad: NO_IDENTIFICADO

*****************************************************************************************************************************
 Titulo:  6 

 la tuberculosi